# Problem statement, data extraction and data tidying procesess

On this Jupyter notebook the problem statement is written to stablish our data science analysis goals. data extraction process will also be performed. Finally, data tidying process will be executed over the retrieved information to structure it and be able to use it properly when performing the analysis.

## Problem statement

The video game industry is one of the most profitable industries worldwide; in fact, in 2020 video games generated more revenue in the United States than movies and sports combined [[1]](https://culturageek.com.ar/en-2020-la-industria-de-los-videojuegos-obtuvo-mas-ganancias-que-el-cine-y-el-deporte-juntos/); however, money is not everything.

In this industry, profits are the harvests for provide enjoyable game experiences to the users which is something very complex to achieve since many factors are involved such as history plot, characters development, balanced difficulty, art design, creative game mechanics, etc. Therefore, video games creators must be aware about the reception of their products because their success depends on it, both financially and in terms of prestige. The way players can feedback is reviewing and scoring the games on official web sites.

The aim of this work is to analyze video games reviews to know which video game genres tend to be well received, as well as to find out which video games platforms and families are the most successful in terms of users satisfaction.

Data about platforms and videogames will be retrieved from [IGDB](https://www.igdb.com/discover) using its [API](https://api-docs.igdb.com/#about).

## Data extraction and data tidying procesess

### Importing dependencies

In [1]:
import requests # To perform API requests
import pandas as pd # To handle data
# To compress the results
import os
import zipfile
from zipfile import ZipFile
# To visualize whole dataframes
pd.set_option("display.max_rows", None, "display.max_columns", None) 

### Getting API credentials

In order to use the [IGDB API](https://api-docs.igdb.com/#about) you need to have a Twitch account and register an application. This is a very easy process, you just need to follow [this instructions](https://api-docs.igdb.com/#account-creation).

Once you have done it, you will need the **Client ID** and the **Client Secret** of your registered Twitch application.

In [1]:
# Write here your Client ID and your Client Secret
clientId = '******************************'
clientSecret = '******************************'

### API authentication

The first step to start using the API is to autenticate to get a valid **Access Token**. As stated in the [instructions](https://api-docs.igdb.com/#authentication), we need to make a `POST` request to https://id.twitch.tv/oauth2/token with our Client ID and Client Secret as parameters.

In [3]:
# Authentication URL
auth_url = 'https://id.twitch.tv/oauth2/token'

# Creating credentials
credentials={
    'client_id':clientId,
    'client_secret': clientSecret,
    'grant_type': 'client_credentials'
}

# Getting access token
response = requests.post(auth_url, params=credentials).json()
try:
    access_token = response['access_token']
    print('Access token recieved')
except:
    print('Error on authentication')
    print(response)
    exit()

Access token recieved


### API requests function

At this point we are able to perform data queries. Now, to avoid code repetition, the following function has been built to perform any request to the API. We only have to specify the endpoint from where the data will be obtained, the fields we are interested in, the limit of results (10 by default, 500 maximum) and the offset which is the n-th element from where the results will start to be obtained.

This is where available [endpoints and fields](https://api-docs.igdb.com/#endpoints) can be consulted.

In [4]:
def query(endpoint, fields, limit, offset=0):
    """
        This function makes a query to the IGDB API via POST request
        
        Params
        ------
        endpoint <str>: The endpoint for an specific resource (resource type).
        fields <str>: Fields of interest of the desired resource.
        limit <int>: Number of results (10 by default, 500 maximum).
        offset <int>: n-th element from where the results will start to be 
                      obtained.
        
        Returns
        -------
        results <list>: List of <dict> objects where each dictionary
                        is a unique result provided by the API.
    """
    # Header
    header = {
        'Client-ID': clientId,
        'Authorization': f'Bearer {access_token}'
    }
    # Base url
    base_url = 'https://api.igdb.com/v4/'
    # Endpoint (resource type)
    endpoint = endpoint
    # parameters
    query = {
        'fields':fields,
        'limit': limit,
        'offset': offset
    }
    # Query url
    url = f'{base_url}{endpoint}'
    
    # Post request execution
    response = requests.post(url, headers=header, params=query)
    # results are converted to dictionaries and are returned
    results = response.json()
    return results

### Getting platforms table

Now we are 100% ready to begin the data extraction process. The first step is to retrieve all the existent video games platforms

In [5]:
# Query to retrieve all existent platforms
endpoint = 'platforms'
fields = 'name,category,generation,platform_family,versions'
limit = 500
results = query(endpoint, fields, limit)
platforms_df = pd.DataFrame(results)

print(len(platforms_df))
platforms_df.head()

181


,id,category,name,versions,generation,platform_family
0,158,6.0,Commodore CDTV,[223],NaN,NaN
1,339,1.0,Sega Pico,[456],4.0,3.0
2,8,1.0,PlayStation 2,"[58, 114]",6.0,1.0
3,39,4.0,iOS,[43],NaN,NaN
4,94,6.0,Commodore Plus/4,[108],NaN,NaN


`category` and `versions` values are encoded. In the [Platform resource documentation](https://api-docs.igdb.com/?shell#platform) the following *Category Enum* is provided, and using it we can map the current category column to its actual values

| name | value |
| ---- | ----- |
| console |	1 |
| arcade | 2 |
| platform | 3 |
| operating_system | 4 |
| portable_console | 5 |
| computer | 6 |



In [6]:
# Category enum provided in the documentation.
platforms_category_enum={
    1:'console',
    2:'arcade',
    3:'platform',
    4:'operating system',
    5:'portable console',
    6:'computer'
}

# Replacing platform category
platforms_df.replace({'category':platforms_category_enum}, inplace=True)

print(len(platforms_df))
platforms_df.head()

181


,id,category,name,versions,generation,platform_family
0,158,computer,Commodore CDTV,[223],NaN,NaN
1,339,console,Sega Pico,[456],4.0,3.0
2,8,console,PlayStation 2,"[58, 114]",6.0,1.0
3,39,operating system,iOS,[43],NaN,NaN
4,94,computer,Commodore Plus/4,[108],NaN,NaN


In order to get the actual `platform_family` column values, we need to perform another query to the `platform_families` endpoint. Then, using the ids of the platform families retrieved, we will be able to replace the current values with the real ones. 

In [7]:
# Getting all platform families
endpoint = 'platform_families'
fields = 'name'
limit = 500
results = query(endpoint, fields, limit)
platforms_families_df = pd.DataFrame(results)
print(len(platforms_families_df))
platforms_families_df

5


,id,name
0,5,Nintendo
1,4,Linux
2,2,Xbox
3,3,Sega
4,1,PlayStation


In [8]:
# Converting platform_families dataframe into a dictionary in order to 
# perform the replacement
platforms_families_dict=platforms_families_df.set_index('id').to_dict()['name']
platforms_df.replace({'platform_family':platforms_families_dict}, inplace=True)
print(len(platforms_df))
platforms_df.head()

181


,id,category,name,versions,generation,platform_family
0,158,computer,Commodore CDTV,[223],NaN,NaN
1,339,console,Sega Pico,[456],4.0,Sega
2,8,console,PlayStation 2,"[58, 114]",6.0,PlayStation
3,39,operating system,iOS,[43],NaN,NaN
4,94,computer,Commodore Plus/4,[108],NaN,NaN


Now, it would be a good idea if we "expand" a row if its `versions` value is a list of two or more elements. After all, each element corresponds to a different version of the platform and it should be treated as a different observation.

In [9]:
platforms_df = platforms_df.explode('versions', ignore_index=True)
print(len(platforms_df))
platforms_df.head(20)

272


,id,category,name,versions,generation,platform_family
0,158,computer,Commodore CDTV,223,NaN,NaN
1,339,console,Sega Pico,456,4.0,Sega
2,8,console,PlayStation 2,58,6.0,PlayStation
3,8,console,PlayStation 2,114,6.0,PlayStation
4,39,operating system,iOS,43,NaN,NaN
5,94,computer,Commodore Plus/4,108,NaN,NaN
6,144,console,AY-3-8710,207,NaN,NaN
7,88,console,Odyssey,101,1.0,NaN
8,88,console,Odyssey,167,1.0,NaN
9,88,console,Odyssey,168,1.0,NaN


Recall `versions` column is encoded, that is, each value corresponds to the id of a `platform_versions` resource instance. So, we need to query all existent **platform_versions** elements to get the actual values of this column 

In [10]:
# All existent platform versions
endpoint = 'platform_versions'
fields = 'name,platform_version_release_dates'
limit = 500
results = query(endpoint, fields, limit)
platform_versions_df = pd.DataFrame(results)

print(len(platform_versions_df))
platform_versions_df.head(10)

275


,id,name,platform_version_release_dates
0,103,Initial version,[552]
1,19,Amiga A 500,[134]
2,282,Switch Lite,[359]
3,139,Super Famicom (SHVC-001),[204]
4,164,Initial version,NaN
5,222,Initial version,NaN
6,283,Wii mini,"[360, 361, 362]"
7,51,Initial version,"[36, 37, 39, 133]"
8,284,Xbox Series X,[363]
9,65,Initial version,"[90, 91, 92]"


Again, it would be a nice move if we explode the dataframe using the `platform_version_release_dates` column in order to obtain the release date of each version. Also the columns will be renamed to avoid confusions

In [11]:
platform_versions_df = platform_versions_df.explode(
    'platform_version_release_dates', 
    ignore_index=True
)

platform_versions_df.rename(
    columns={"id": "id_version", 
             'name':'version_name', 
             'platform_version_release_dates':'version_release_date'},
    inplace=True
)
print(len(platform_versions_df))
platform_versions_df.head(10)

436


,id_version,version_name,version_release_date
0,103,Initial version,552
1,19,Amiga A 500,134
2,282,Switch Lite,359
3,139,Super Famicom (SHVC-001),204
4,164,Initial version,NaN
5,222,Initial version,NaN
6,283,Wii mini,360
7,283,Wii mini,361
8,283,Wii mini,362
9,51,Initial version,36


The `version_release_date` values are ids of **platform_version_release_dates** resources. We need to repeat the process to obtain all instances of the dates resources and map the current values to their actual release date using the id of the *platform_version_release_dates* resources

In [12]:
# All existent platform versions release dates
endpoint = 'platform_version_release_dates'
fields = 'category,human,date,region'
limit = 500
results = query(endpoint, fields, limit)
platform_version_release_date_df = pd.DataFrame(results)

print(len(platform_version_release_date_df))
platform_version_release_date_df.head()

377


,id,category,date,human,region
0,555,2,567907200,1987,8
1,338,0,1189555200,"Sep 12, 2007",3
2,334,0,1125532800,"Sep 01, 2005",7
3,331,0,1415923200,"Nov 14, 2014",3
4,333,0,1389830400,"Jan 16, 2014",7


`id` and `category` columns will be renamed with more appropriate names

In [13]:
platform_version_release_date_df.rename(
    columns={'id':'id_release_date', 
             'category':'date_format'}, 
    inplace=True
)

print(len(platform_version_release_date_df))
platform_version_release_date_df.head()

377


,id_release_date,date_format,date,human,region
0,555,2,567907200,1987,8
1,338,0,1189555200,"Sep 12, 2007",3
2,334,0,1125532800,"Sep 01, 2005",7
3,331,0,1415923200,"Nov 14, 2014",3
4,333,0,1389830400,"Jan 16, 2014",7


`date_format` indicates in which format `date` column values should be parsed. In the [Platform version release date resource documentation](https://api-docs.igdb.com/?shell#platform-version-release-date) the following **category enum** is provided with which we can identify the actual date format:

| Name      | Datetime |
| ----------- | ----------- |
| YYYYMMMMDD      | 0      |
| YYYYMMMM   | 1        |
| YYYY   | 2       |
| YYYYQ1  | 3        |
| YYYYQ2   | 4        |
| YYYYQ3   | 5        |
| YYYYQ4   | 6        |
| TBD   | 7        |

There are some special cases and they will be handled carefully. First, we don't care if version was released in 1st, 2nd or whichever quarter of a year, in this case we only care the year itself. The second special case is when we find a **TBD** (to be determined) date. In those cases, `NaN` will be set. 

All `date_format` values will be written with the default timestamps formats, but this formats will be established based on the category enum provided by IGDB and the special cases we have discussed. 

In [14]:
date_formats_enum = {
    0: '%Y-%m-%d',
    1: '%Y-%m',
    2: '%Y',
    3: '%Y',
    4: '%Y',
    5: '%Y',
    6: '%Y',
    7: 'TBD'
}

platform_version_release_date_df.replace(
    {'date_format':date_formats_enum}, 
    inplace=True
)
print(len(platform_version_release_date_df))
platform_version_release_date_df.head()

377


,id_release_date,date_format,date,human,region
0,555,%Y,567907200,1987,8
1,338,%Y-%m-%d,1189555200,"Sep 12, 2007",3
2,334,%Y-%m-%d,1125532800,"Sep 01, 2005",7
3,331,%Y-%m-%d,1415923200,"Nov 14, 2014",3
4,333,%Y-%m-%d,1389830400,"Jan 16, 2014",7


Using the `date_format` values we can parse the Unix timestamps in `date` column

In [15]:
# dummy values to the new release_date column. This is done just to create the
# new column and later can compute with it.
platform_version_release_date_df['release_date'] = 0

# Parsing unix timestamp to pandas datetime
for i in range(len(platform_version_release_date_df)):
    # Getting date format
    date_format = platform_version_release_date_df.iloc[i,1]
    # Checking date format to decide whether None or datetime should be set
    if date_format == 'TBD':
        platform_version_release_date_df.iloc[i,5] = None
    else:
        platform_version_release_date_df.iloc[i,5]=pd.to_datetime(
            platform_version_release_date_df.iloc[i,2], 
            unit='s', 
            origin='unix'
        )
        # Changing the format according to the date format provided by IGDB
        platform_version_release_date_df.iloc[i,5]=platform_version_release_date_df.iloc[i,5].strftime(
            date_format
        )

print(len(platform_version_release_date_df))
platform_version_release_date_df.head()

377


,id_release_date,date_format,date,human,region,release_date
0,555,%Y,567907200,1987,8,1987
1,338,%Y-%m-%d,1189555200,"Sep 12, 2007",3,2007-09-12
2,334,%Y-%m-%d,1125532800,"Sep 01, 2005",7,2005-09-01
3,331,%Y-%m-%d,1415923200,"Nov 14, 2014",3,2014-11-14
4,333,%Y-%m-%d,1389830400,"Jan 16, 2014",7,2014-01-16


Now `date_format`, `date` and `human` columns are no more needed, so we delete them

In [16]:
platform_version_release_date_df.drop(
    columns=['date_format', 
             'date', 
             'human'], 
    inplace=True
)

print(len(platform_version_release_date_df))
platform_version_release_date_df.head()

377


,id_release_date,region,release_date
0,555,8,1987
1,338,3,2007-09-12
2,334,7,2005-09-01
3,331,3,2014-11-14
4,333,7,2014-01-16


For the `region` column is also provided the following **Region enum** to identify the actual region where the version was released

| name | value |
| ---- | ----- |
| europe | 1 |
| north_america | 2 |
| australia | 3 |
| new_zealand | 4 |
| japan | 5 |
| china | 6 |
| asia | 7 |
| worldwide | 8 |
| korea | 9 |
| brazil | 10 |

In [17]:
region_enum = {
    1:'europe',
    2:'north_america',
    3:'australia',
    4:'new_zealand',
    5:'japan',
    6:'china',
    7:'asia',
    8:'worldwide',
    9:'korea',
    10:'brazil'
}

platform_version_release_date_df.replace({'region':region_enum}, inplace=True)
print(len(platform_version_release_date_df))
platform_version_release_date_df.head()

377


,id_release_date,region,release_date
0,555,worldwide,1987
1,338,australia,2007-09-12
2,334,asia,2005-09-01
3,331,australia,2014-11-14
4,333,asia,2014-01-16


Now, we have the complete information about versions and their release dates segmented in two dataframes which can be joined using `version_release_date` column of **platforms_versions_df** and `id_release_date` column of **platform_version_release_date_df** 

In [18]:
print(f'VERSIONS DATAFRAME ({len(platform_versions_df)} rows)')
platform_versions_df.head()

VERSIONS DATAFRAME (436 rows)


,id_version,version_name,version_release_date
0,103,Initial version,552
1,19,Amiga A 500,134
2,282,Switch Lite,359
3,139,Super Famicom (SHVC-001),204
4,164,Initial version,NaN


In [19]:
print(f'VERSIONS RELEASE DATES DATAFRAME ({len(platform_version_release_date_df)} rows)')
platform_version_release_date_df.head()

VERSIONS RELEASE DATES DATAFRAME (377 rows)


,id_release_date,region,release_date
0,555,worldwide,1987
1,338,australia,2007-09-12
2,334,asia,2005-09-01
3,331,australia,2014-11-14
4,333,asia,2014-01-16


In [20]:
versions_df = pd.merge(
    platform_versions_df,
    platform_version_release_date_df,
    left_on='version_release_date',
    right_on='id_release_date', 
    how='left'
)
print(len(versions_df))
versions_df.head()

436


,id_version,version_name,version_release_date,id_release_date,region,release_date
0,103,Initial version,552,552.0,worldwide,1977-01
1,19,Amiga A 500,134,134.0,worldwide,1987-10
2,282,Switch Lite,359,359.0,worldwide,2019-09-20
3,139,Super Famicom (SHVC-001),204,204.0,japan,1990-11-21
4,164,Initial version,NaN,NaN,NaN,NaN


We don't need the `version_release_date` and `id_release_date` columns anymore

In [21]:
versions_df.drop(
    columns=['version_release_date', 
             'id_release_date'], 
    inplace=True
)

print(f'VERSIONS DATAFRAME ({len(versions_df)} rows)')
versions_df.head()

VERSIONS DATAFRAME (436 rows)


,id_version,version_name,region,release_date
0,103,Initial version,worldwide,1977-01
1,19,Amiga A 500,worldwide,1987-10
2,282,Switch Lite,worldwide,2019-09-20
3,139,Super Famicom (SHVC-001),japan,1990-11-21
4,164,Initial version,NaN,NaN


Finally, we have the versions and their release dates in a single table. Now we can join this new dataframe to the **platforms_df** 

In [22]:
print(f'PLATFORMS DATAFRAME ({len(platforms_df)} rows)')
platforms_df.head()

PLATFORMS DATAFRAME (272 rows)


,id,category,name,versions,generation,platform_family
0,158,computer,Commodore CDTV,223,NaN,NaN
1,339,console,Sega Pico,456,4.0,Sega
2,8,console,PlayStation 2,58,6.0,PlayStation
3,8,console,PlayStation 2,114,6.0,PlayStation
4,39,operating system,iOS,43,NaN,NaN


In [23]:
platforms_df = pd.merge(
    versions_df, 
    platforms_df, 
    left_on='id_version', 
    right_on='versions', 
    how='right'
)

print(len(platforms_df))
platforms_df.head()

433


,id_version,version_name,region,release_date,id,category,name,versions,generation,platform_family
0,223,Initial version,north_america,1991-03-03,158,computer,Commodore CDTV,223,NaN,NaN
1,223,Initial version,europe,1991-11-14,158,computer,Commodore CDTV,223,NaN,NaN
2,223,Initial version,australia,1991-07-12,158,computer,Commodore CDTV,223,NaN,NaN
3,456,Initial version,NaN,NaN,339,console,Sega Pico,456,4.0,Sega
4,58,Initial version,japan,2000-03-04,8,console,PlayStation 2,58,6.0,PlayStation


`id_version` and `versions` columns will be removed as they are no more needed. `name` column will be renamed to `platform`, `version_name` column will be renamed to `version` and all the remaining columns will be sorted

In [24]:
# Removing useless columns
platforms_df.drop(columns=['id_version', 'versions'], inplace=True)
# Renaming name column
platforms_df.rename(
    columns={
        'name':'platform', 
        'version_name':'version'
    }, 
    inplace=True
)
# Sorting the columns
platforms_df = platforms_df[['id','platform','version','category',
                             'generation','platform_family','region',
                             'release_date']
                           ]
print(len(platforms_df))
platforms_df.head()

433


,id,platform,version,category,generation,platform_family,region,release_date
0,158,Commodore CDTV,Initial version,computer,NaN,NaN,north_america,1991-03-03
1,158,Commodore CDTV,Initial version,computer,NaN,NaN,europe,1991-11-14
2,158,Commodore CDTV,Initial version,computer,NaN,NaN,australia,1991-07-12
3,339,Sega Pico,Initial version,console,4.0,Sega,NaN,NaN
4,8,PlayStation 2,Initial version,console,6.0,PlayStation,japan,2000-03-04


### Getting video games tables

Now, the task is to retrieve all games for all existent platforms. To achieve this goal, the next function was built

In [25]:
def getAllGames(platform_id):
    """
        This function retireves all video games stored in IGDB databases for a
        given platform.
        
        Params
        ------
        platform_id <str>: Id of the platform.
        
        Returns
        -------
        df <pd.DataFrame>: Dataframe containing all the videogames of the given
                           platform.
    """
    # Flag to indicate when all games were retrieved
    completed = False
    # Dataframe to save the results
    df = pd.DataFrame()
    # Query parameters
    endpoint = 'games'
    variables = ['category','first_release_date','game_modes','genres',
                 'name','platforms','total_rating']

    fields = f"{','.join(variables)}; where platforms = [{platform_id}]"
    limit = 500
    # The api only retireve 500 results at a time, so we need the offset to 
    # specify from which item start to retrieve results
    offset = 0 
    # Loop to retrieve all games of the platform
    while not completed:
        # Getting the results from the api and converting it to a dataframe
        results = pd.DataFrame(query(endpoint, fields, limit, offset))
        # Concatenate the current results with the previous results
        df = pd.concat([df,results], ignore_index=True)
        # If current results are empty or there are less than 500 items, we have
        # retrieved every single game
        if results.empty or len(results) < 500:
            completed = True
        
        # If there are remaining results, then adjust the next offset
        offset += 500
    
    return df
        

Now, we just need to run `getAllGames()` function for each unique platform within **platform_df** and will save all games in a single dataframe.

**NOTE**: This process may take some minutes to finish.

In [26]:
# Getting all unique platforms
unique_platforms = platforms_df.platform.unique()
# Support variables to have some feedback and get an idea of 
# this procedure progress
n = len(unique_platforms)
i = 1
# Dataframe where all games will be stored
games_df = pd.DataFrame()

# Retrieving all games 
for platform in unique_platforms:    
    # Printing progress
    print(f'{i}/{n}', end='\r')
    # Getting the id of the current platform
    platform_id = platforms_df[platforms_df.platform == platform].iloc[0,0]
    # Using the previous function to retrieve all games
    df = getAllGames(platform_id)
    # Concatenating the current game with the dataframe of all games
    games_df = pd.concat([games_df, df], ignore_index=True)
    i += 1

We have retrieved all games, we will sort and rename the columns

In [27]:
# Sorting the columns 
cols = ['id','name','platforms','first_release_date',
        'genres','category','game_modes',
        'total_rating'
       ]
games_df = games_df[cols]

# Renaming the columns
games_df.rename(columns={'platforms':'platform', 
                         'first_release_date':'release_date',
                         'genres':'genre'}, 
                inplace=True
               )

In [28]:
print(len(games_df))
games_df.tail()

181332


,id,name,platform,release_date,genre,category,game_modes,total_rating
181327,181314,Drawing Evolution,[389],1.587686e+09,"[26, 32, 35]",0,[2],NaN
181328,180726,Tower of Babel,[389],NaN,[33],0,[2],NaN
181329,180592,The Neighborhood,[389],NaN,[16],0,"[1, 2, 3]",NaN
181330,180701,Racing Wars,[389],NaN,[10],0,"[1, 2]",NaN
181331,180736,Mucho Muscle,[389],NaN,[33],0,"[2, 3]",NaN


Now, we need to replace encoded columns to their real values, note that we are requesting for several fields. Variables `platform`, `genre` and `game_modes` are lists since each game could have multiple game modes, involved companies and so on. For this reason we need to *explode* all this columns.

In [29]:
# Exploiting the columns
explode_columns = ['platform', 'genre', 'game_modes']
for c in explode_columns:
    games_df = games_df.explode(c, ignore_index = True)
print(len(games_df))
games_df.head()

1773379


,id,name,platform,release_date,genre,category,game_modes,total_rating
0,66345,Snoopy: The Cool Computer Game,13,631065600.0,9,0,1,NaN
1,66345,Snoopy: The Cool Computer Game,13,631065600.0,31,0,1,NaN
2,66345,Snoopy: The Cool Computer Game,16,631065600.0,9,0,1,NaN
3,66345,Snoopy: The Cool Computer Game,16,631065600.0,31,0,1,NaN
4,66345,Snoopy: The Cool Computer Game,25,631065600.0,9,0,1,NaN


We will start replacing the values in `category` column. In the documentation the following **category enum** is provided. It will be used to perform the replacement

| name | value | 
| ---- | ----- |
| main_game | 0 |
| dlc_addon	| 1 |
| expansion | 2 |
| bundle | 3 |
| standalone_expansion | 4 |
| mod |	5 |
| episode |	6 |
| season | 7 |
| remake | 8 |
| remaster | 9 |
| expanded_game | 10 |
| port | 11 |
| fork | 12 |

In [30]:
# Category dictionary
category_dict = {0:'main_game',1:'dlc_addon',2:'expansion',3:'bundle',
                 4:'standalone_expansion',5:'mod',6:'episode',7:'season',
                 8:'remake',9:'remaster',10:'expanded_game',11:'port',
                 12:'fork'
                }
# category column replacement
games_df.replace({'category':category_dict}, inplace=True)
print(len(games_df))
games_df.head()

1773379


,id,name,platform,release_date,genre,category,game_modes,total_rating
0,66345,Snoopy: The Cool Computer Game,13,631065600.0,9,main_game,1,NaN
1,66345,Snoopy: The Cool Computer Game,13,631065600.0,31,main_game,1,NaN
2,66345,Snoopy: The Cool Computer Game,16,631065600.0,9,main_game,1,NaN
3,66345,Snoopy: The Cool Computer Game,16,631065600.0,31,main_game,1,NaN
4,66345,Snoopy: The Cool Computer Game,25,631065600.0,9,main_game,1,NaN


Now, we will replace `platform` column ids with the real name of the platforms. This will be done using the **platform_df**

In [31]:
# Converting 'id' and 'platform' columns of platforms_df into into a dictionary
# in order to perform the replacement
platforms_dict = platforms_df[
    [
        'id',
        'platform'
    ]
].set_index('id').to_dict()['platform']
games_df.replace({'platform':platforms_dict}, inplace=True)
print(len(games_df))
games_df.head()

1773379


,id,name,platform,release_date,genre,category,game_modes,total_rating
0,66345,Snoopy: The Cool Computer Game,PC DOS,631065600.0,9,main_game,1,NaN
1,66345,Snoopy: The Cool Computer Game,PC DOS,631065600.0,31,main_game,1,NaN
2,66345,Snoopy: The Cool Computer Game,Amiga,631065600.0,9,main_game,1,NaN
3,66345,Snoopy: The Cool Computer Game,Amiga,631065600.0,31,main_game,1,NaN
4,66345,Snoopy: The Cool Computer Game,Amstrad CPC,631065600.0,9,main_game,1,NaN


Now we will continune with `genre` column. We will use the previous strategy of retrieve all **genre** elements.

In [32]:
# All existent platform versions release dates
endpoint = 'genres'
fields = 'name'
limit = 500
results = query(endpoint, fields, limit)
genres_df = pd.DataFrame(results)
print(len(genres_df))
genres_df.head()

23


,id,name
0,4,Fighting
1,5,Shooter
2,7,Music
3,8,Platform
4,9,Puzzle


In [33]:
# Converting genres_Df into into a dictionary
# in order to perform the replacement
genres_dict = genres_df.set_index('id').to_dict()['name']

games_df.replace({'genre':genres_dict}, inplace=True)
print(len(games_df))
games_df.head()

1773379


,id,name,platform,release_date,genre,category,game_modes,total_rating
0,66345,Snoopy: The Cool Computer Game,PC DOS,631065600.0,Puzzle,main_game,1,NaN
1,66345,Snoopy: The Cool Computer Game,PC DOS,631065600.0,Adventure,main_game,1,NaN
2,66345,Snoopy: The Cool Computer Game,Amiga,631065600.0,Puzzle,main_game,1,NaN
3,66345,Snoopy: The Cool Computer Game,Amiga,631065600.0,Adventure,main_game,1,NaN
4,66345,Snoopy: The Cool Computer Game,Amstrad CPC,631065600.0,Puzzle,main_game,1,NaN


For `game_modes` column the same process will be repeated.

In [34]:
# Retrieving all existent game modes 
endpoint = 'game_modes' 
fields = 'name'
limit = 500
results = query(endpoint, fields, limit)
game_modes_df = pd.DataFrame(results)
print(len(genres_df))
game_modes_df.head()

23


,id,name
0,1,Single player
1,2,Multiplayer
2,3,Co-operative
3,4,Split screen
4,5,Massively Multiplayer Online (MMO)


In [35]:
# Converting game_modes_df into into a dictionary
# in order to perform the replacement
game_modes_dict = game_modes_df.set_index('id').to_dict()['name']

games_df.replace({'game_modes':game_modes_dict}, inplace=True)
print(len(games_df))
games_df.head()

1773379


,id,name,platform,release_date,genre,category,game_modes,total_rating
0,66345,Snoopy: The Cool Computer Game,PC DOS,631065600.0,Puzzle,main_game,Single player,NaN
1,66345,Snoopy: The Cool Computer Game,PC DOS,631065600.0,Adventure,main_game,Single player,NaN
2,66345,Snoopy: The Cool Computer Game,Amiga,631065600.0,Puzzle,main_game,Single player,NaN
3,66345,Snoopy: The Cool Computer Game,Amiga,631065600.0,Adventure,main_game,Single player,NaN
4,66345,Snoopy: The Cool Computer Game,Amstrad CPC,631065600.0,Puzzle,main_game,Single player,NaN


Now, we just need to parse the `release_date` column. This is an easy cake since those values are Unix timestamps

In [36]:
games_df.release_date = pd.to_datetime(games_df.release_date, 
                                       unit='s', 
                                       origin='unix', 
                                       errors='coerce'
)
print(len(games_df))
games_df.head()

1773379


,id,name,platform,release_date,genre,category,game_modes,total_rating
0,66345,Snoopy: The Cool Computer Game,PC DOS,1989-12-31,Puzzle,main_game,Single player,NaN
1,66345,Snoopy: The Cool Computer Game,PC DOS,1989-12-31,Adventure,main_game,Single player,NaN
2,66345,Snoopy: The Cool Computer Game,Amiga,1989-12-31,Puzzle,main_game,Single player,NaN
3,66345,Snoopy: The Cool Computer Game,Amiga,1989-12-31,Adventure,main_game,Single player,NaN
4,66345,Snoopy: The Cool Computer Game,Amstrad CPC,1989-12-31,Puzzle,main_game,Single player,NaN


And that's it!. Finally, we have all data about each platform and their video games. Now, `platforms_df` and `games_df` will be compressed and stored

In [37]:
# Creating the data folder if it does not exists
if not os.path.exists('./raw_data'):
    os.makedirs('./raw_data')

# Storing the csvs
platforms_df.to_csv('./raw_data/platforms.csv', index=False)
games_df.to_csv('./raw_data/games.csv', index=False)

# Compressing the csvs
with ZipFile('./raw_data/platforms.zip', 'w') as zip:
    zip.write('./raw_data/platforms.csv', 
              compress_type=zipfile.ZIP_DEFLATED, 
              compresslevel=9
             )

with ZipFile('./raw_data/games.zip', 'w') as zip:
    zip.write('./raw_data/games.csv', 
              compress_type=zipfile.ZIP_DEFLATED, 
              compresslevel=9
             )
    
# Removing the csvs
os.remove('./raw_data/platforms.csv')
os.remove('./raw_data/games.csv')

## Data dictionaries

And, last but not least, a data dictionaries for both platforms and games csvs will be created

### Platforms dataframe dictionary

In [38]:
cols = [
    'id', 
    'platform', 
    'version', 
    'category', 
    'generation', 
    'platform_family', 
    'region', 
    'release_date'
]

descriptions = [
    "Platform id on 'platforms' endpoint at IGDB database",
    "Name of the platform",
    "Name of the platform version",
    "Type of platform",
    "Video games generation in which the platform was released",
    "Platform publisher",
    "Region of the World where the platform was released",
    "Release date of the platform"
]

units = [
    "Integer",
    "String",
    "String",
    "String",
    "Integer",
    "String",
    "String",
    "Datetime"
]

platforms_data_dict = pd.DataFrame({
    "Column name": cols,
    "Description": descriptions,
    "Units": units
})

platforms_data_dict


,Column name,Description,Units
0,id,Platform id on 'platforms' endpoint at IGDB da...,Integer
1,platform,Name of the platform,String
2,version,Name of the platform version,String
3,category,Type of platform,String
4,generation,Video games generation in which the platform w...,Integer
5,platform_family,Platform publisher,String
6,region,Region of the World where the platform was rel...,String
7,release_date,Release date of the platform,Datetime


### Video games dataframe dictionary

In [39]:
cols = [
    'id',
    'name',
    'platform',
    'release_date',
    'genre',
    'category',
    'game_modes',
    'total_rating',
]

descriptions = [
    "Video game id on 'games' endpoint at IGDB database",
    "Name of the video game",
    "Platfrom on which the videogame was released",
    "Release date",
    "Genre of the videogame",
    "Type of the product",
    "Available game modes",
    "User's rating (0-100)"
]

units = [
    "Integer",
    "String",
    "String",
    "Datetime",
    "String",
    "String",
    "String",
    "Float"
]

games_data_dict = pd.DataFrame({
    "Column name": cols,
    "Description": descriptions,
    "Units": units
})

games_data_dict

,Column name,Description,Units
0,id,Video game id on 'games' endpoint at IGDB data...,Integer
1,name,Name of the video game,String
2,platform,Platfrom on which the videogame was released,String
3,release_date,Release date,Datetime
4,genre,Genre of the videogame,String
5,category,Type of the product,String
6,game_modes,Available game modes,String
7,total_rating,User's rating (0-100),Float


Now, those dictionaries will be stores

In [40]:
# Creating the data dictionaries folder if it does not exists
if not os.path.exists('./data_dicts'):
    os.makedirs('./data_dicts')

# Storing the csvs
platforms_data_dict.to_csv('./data_dicts/platforms_dictionary.csv', index=False)
games_data_dict.to_csv('./data_dicts/games_dictionary.csv', index=False)